<a href="https://colab.research.google.com/github/JesseDiGiacomo/WebScraping/blob/main/XCBrasilWebScrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The main goal of this job was to collect from the national database site (XCBrasil.com.br) all the official flights from the city of Petropolis / RJ and analyze them. 

To start the web scraping we analyzed the XCBrasil.com.br and discovered we could filter the flights by take-off location.
The city of Petropolis has three (3) take-offs. They are logged into fifteen (15) different take-offs due to misspelling errors. 

We picked one of the registered take-offs to test the best way to proceed.

First, conect to Google Drive so we can store the data acquired.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Them stablish where we want to store it.



In [2]:
path = "/content/drive/MyDrive/XCBrasil_flights/"

Now we import the libraries to work with

In [3]:
# Imports
import os
import csv
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import scipy
import statsmodels
import statsmodels.api as sm
import matplotlib.patches as mpatches
from statsmodels.formula.api import ols
from tabulate import tabulate
from pandas_profiling import ProfileReport

%matplotlib inline

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Then we create the CSV file with the header and open it.

In [4]:
# Creates the Final Data file
outputfile = open((path + 'FlightsData.csv'), 'a')
writer = csv.writer(outputfile, delimiter=',',quotechar='"', quoting=csv.QUOTE_ALL)

# Adds Header
writer.writerow(['Date', 'Pilot',	'Location', 'Duration', 'Straight Distance', 	'OLC KM', 'OLC Points'])

80

#Testing

Stabilish the URL and start scraping and testing

In [5]:
# XCBrasil homepage
base_url = 'http://xcbrasil.com.br'
xcbrasil_url = 'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8892'

In [6]:
lst = []

condition = True

while condition: 
  # Use requests to retrieve data from a given URL
  xcbrasil_response = requests.get(xcbrasil_url)

  # Parse the whole HTML page using BeautifulSoup
  xcbrasil_soup = BeautifulSoup(xcbrasil_response.text, 'html.parser')

  # Title of the parsed page
  xcbrasil_soup.title

  # Reading table
  flights = xcbrasil_soup.find('table', {'class':'listTable'})

  for row in flights.find_all('tr')[1:-1]:
      s = pd.Series([data.text for data in row.find_all('td')])
      lst.append(s)
  
  # Next Page
  pg = xcbrasil_soup.find('div', {'class':'pagesDiv'})
  active_pg = pg.find('div', {'class':'activePageNum'})
  
  # Test if is Blank
  if active_pg is None:
    condition = False
  if active_pg.findNextSibling('a') is None:
    condition = False
  else:
    next_url = base_url + active_pg.findNextSibling('a').get("href")
    xcbrasil_url = next_url

In [7]:
# Concatenate the Pandas Series in a DataFrame
data = pd.concat(lst, axis=1).T

In [8]:
data.tail()

,0,1,2,3,4,5,6,7,8,9
3043,3106,17/01/2006,Daniel Wenna\nPetropolis City (Simeria takeof...,0:33,4.7 km,6.7 km,10.08,,,
3044,3107,15/09/2001,Artur Pereira\nPetropolis City (Simeria take...,1:43,5.2 km,18.5 km,27.76,,,
3045,3108,09/06/2001,Artur Pereira\nPetropolis City (Simeria take...,1:47,7.0 km,19.4 km,29.04,,,
3046,3109,06/05/2001,Artur Pereira\nPetropolis City (Simeria take...,1:48,25.3 km,32.3 km,48.39,,,
3047,3110,13/05/2000,Artur Pereira\nPetropolis City (Simeria take...,1:13,13.5 km,18.8 km,28.23,,,


We need to delete some columns we don't need and split the ones with information regarding the pilot and take-off site. Then we create the CSV file with the header and open it.

In [9]:
data = data.drop(columns=[0,7,8,9])
data.head()

,1,2,3,4,5,6
0,07/06/2022,henrique pessoa\nPetropolis City (Simeria tak...,0:39,5.4 km,8.0 km,12.04
1,07/06/2022,Victor Affonso Ferreira Zarlotti\nPetropolis ...,0:30,4.8 km,7.1 km,10.70
2,07/06/2022,Rafael Baroni\nPetropolis City (Simeria takeo...,0:13,4.6 km,5.4 km,8.05
3,05/06/2022,Victor Affonso Ferreira Zarlotti\nPetropolis ...,0:48,6.1 km,9.7 km,14.62
4,22/05/2022,Vagner Garcia\nPetropolis City (Simeria takeo...,0:48,4.7 km,11.6 km,17.34


In [10]:
data.columns = ['Date', 'Pilot and Takeoff', 'Duration', 'Straight Distance', 'OLC KM', 'OLC Points']

In [11]:
data.head()

,Date,Pilot and Takeoff,Duration,Straight Distance,OLC KM,OLC Points
0,07/06/2022,henrique pessoa\nPetropolis City (Simeria tak...,0:39,5.4 km,8.0 km,12.04
1,07/06/2022,Victor Affonso Ferreira Zarlotti\nPetropolis ...,0:30,4.8 km,7.1 km,10.70
2,07/06/2022,Rafael Baroni\nPetropolis City (Simeria takeo...,0:13,4.6 km,5.4 km,8.05
3,05/06/2022,Victor Affonso Ferreira Zarlotti\nPetropolis ...,0:48,6.1 km,9.7 km,14.62
4,22/05/2022,Vagner Garcia\nPetropolis City (Simeria takeo...,0:48,4.7 km,11.6 km,17.34


In [12]:
df2 = data['Pilot and Takeoff'].str.split('\n', expand=True)
df2.drop(columns=[2], inplace=True)
df2.columns = ['Pilot', 'Location']
df2.head()

,Pilot,Location
0,henrique pessoa,Petropolis City (Simeria takeoff) - BR [~0.6]
1,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR
2,Rafael Baroni,Petropolis City (Simeria takeoff) - BR [~0.6]
3,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR
4,Vagner Garcia,Petropolis City (Simeria takeoff) - BR


In [13]:
data = data.join(df2)

In [14]:
data.head()

,Date,Pilot and Takeoff,Duration,Straight Distance,OLC KM,OLC Points,Pilot,Location
0,07/06/2022,henrique pessoa\nPetropolis City (Simeria tak...,0:39,5.4 km,8.0 km,12.04,henrique pessoa,Petropolis City (Simeria takeoff) - BR [~0.6]
1,07/06/2022,Victor Affonso Ferreira Zarlotti\nPetropolis ...,0:30,4.8 km,7.1 km,10.70,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR
2,07/06/2022,Rafael Baroni\nPetropolis City (Simeria takeo...,0:13,4.6 km,5.4 km,8.05,Rafael Baroni,Petropolis City (Simeria takeoff) - BR [~0.6]
3,05/06/2022,Victor Affonso Ferreira Zarlotti\nPetropolis ...,0:48,6.1 km,9.7 km,14.62,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR
4,22/05/2022,Vagner Garcia\nPetropolis City (Simeria takeo...,0:48,4.7 km,11.6 km,17.34,Vagner Garcia,Petropolis City (Simeria takeoff) - BR


In [15]:
data = data.drop(columns=['Pilot and Takeoff'])
data.head()

,Date,Duration,Straight Distance,OLC KM,OLC Points,Pilot,Location
0,07/06/2022,0:39,5.4 km,8.0 km,12.04,henrique pessoa,Petropolis City (Simeria takeoff) - BR [~0.6]
1,07/06/2022,0:30,4.8 km,7.1 km,10.70,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR
2,07/06/2022,0:13,4.6 km,5.4 km,8.05,Rafael Baroni,Petropolis City (Simeria takeoff) - BR [~0.6]
3,05/06/2022,0:48,6.1 km,9.7 km,14.62,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR
4,22/05/2022,0:48,4.7 km,11.6 km,17.34,Vagner Garcia,Petropolis City (Simeria takeoff) - BR


In [16]:
data = data[['Date', 'Pilot',	'Location', 'Duration', 'Straight Distance', 	'OLC KM', 'OLC Points']]
data.head()

,Date,Pilot,Location,Duration,Straight Distance,OLC KM,OLC Points
0,07/06/2022,henrique pessoa,Petropolis City (Simeria takeoff) - BR [~0.6],0:39,5.4 km,8.0 km,12.04
1,07/06/2022,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR,0:30,4.8 km,7.1 km,10.70
2,07/06/2022,Rafael Baroni,Petropolis City (Simeria takeoff) - BR [~0.6],0:13,4.6 km,5.4 km,8.05
3,05/06/2022,Victor Affonso Ferreira Zarlotti,Petropolis City (Simeria takeoff) - BR,0:48,6.1 km,9.7 km,14.62
4,22/05/2022,Vagner Garcia,Petropolis City (Simeria takeoff) - BR,0:48,4.7 km,11.6 km,17.34


Now we build a routine to scrap all the fifteen takeoffs registered for Petropolis/RJ, save them to a CSV file, and close it.

# Web Scraping Data

In [17]:
#Scrapping others take offs
base_url = 'http://xcbrasil.com.br'
takeoff_url_list = [
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8892',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9222',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9236',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9276',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8897',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8960',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8900',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9333',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:13051',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9495',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9283',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:9275',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8945',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:11051',
                    'http://xcbrasil.com.br/tracks/world/alltimes/brand:all,cat:0,class:all,xctype:all,club:all,pilot:all,takeoff:8892'
                    ]
                
for takeoff in takeoff_url_list:
  xcbrasil_url = takeoff

  lst = []

  condition = True

  while condition: 
    # Use requests to retrieve data from a given URL
    xcbrasil_response = requests.get(xcbrasil_url)

    # Parse the whole HTML page using BeautifulSoup
    xcbrasil_soup = BeautifulSoup(xcbrasil_response.text, 'html.parser')

    # Title of the parsed page
    xcbrasil_soup.title

    # Reading table
    flights = xcbrasil_soup.find('table', {'class':'listTable'})

    for row in flights.find_all('tr')[1:-1]:
        s = pd.Series([data.text for data in row.find_all('td')])
        lst.append(s)
    
    # Next Page
    pg = xcbrasil_soup.find('div', {'class':'pagesDiv'})
    active_pg = pg.find('div', {'class':'activePageNum'})
    
    
    # Test if is Blank
    if active_pg is None:
      condition = False
    else:
      if active_pg.findNextSibling('a') is None:
        condition = False
      else:
        next_url = base_url + active_pg.findNextSibling('a').get("href")
        xcbrasil_url = next_url

  # Concatenate the Pandas Series in a DataFrame
  data = pd.concat(lst, axis=1).T

  # Dropping non usable columns
  data = data.drop(columns=[0,7,8,9])

  # Naming Columns
  data.columns = ['Date', 'Pilot and Takeoff', 'Duration', 'Straight Distance', 'OLC KM', 'OLC Points']

  # Spliting Pilot and Takeoff
  df2 = data['Pilot and Takeoff'].str.split('\n', expand=True)
  df2.drop(columns=[2], inplace=True)
  df2.columns = ['Pilot', 'Location']
  df2.head()
  data = data.join(df2)
  data = data.drop(columns=['Pilot and Takeoff'])

  # Reorganizing Columns
  data = data[['Date', 'Pilot',	'Location', 'Duration', 'Straight Distance', 	'OLC KM', 'OLC Points']]

  # Writing to CSV
  data.to_csv(outputfile, index=False, header=False)

In [18]:
outputfile.close()

# Cleaning Data